In [ ]:
import pandas as pd

In [ ]:
weather_dir = 'I:/Weather Data/'

In [ ]:
daily_tmin_df = pd.read_fwf(f'{weather_dir}/dly-tmin-normal.txt', header=None)
daily_tmin_df.columns = ['STNID','MONTH'] + [i for i in range(1,32)]
daily_tmin_df = pd.melt(daily_tmin_df, id_vars = ['STNID','MONTH'], value_vars = [i for i in range(1,32)])
daily_tmin_df.columns = ['STNID','MONTH','DAY','VALUE']
daily_tmin_df.head()

In [ ]:
daily_tmin_df = daily_tmin_df[daily_tmin_df['VALUE'] != '-8888']
daily_tmin_df['FLAG'] = daily_tmin_df['VALUE'].map(lambda x: x[-1])
daily_tmin_df['VALUE'] = daily_tmin_df['VALUE'].map(lambda x: int(x[0:-1])/10)
daily_tmin_df

In [ ]:
daily_tmax_df = pd.read_fwf(f'{weather_dir}/dly-tmax-normal.txt', header=None)
daily_tmax_df.columns = ['STNID','MONTH'] + [i for i in range(1,32)]
daily_tmax_df = pd.melt(daily_tmax_df, id_vars = ['STNID','MONTH'], value_vars = [i for i in range(1,32)])
daily_tmax_df.columns = ['STNID','MONTH','DAY','VALUE']

daily_tmax_df = daily_tmax_df[daily_tmax_df['VALUE'] != '-8888']
daily_tmax_df['FLAG'] = daily_tmax_df['VALUE'].map(lambda x: x[-1])
daily_tmax_df['VALUE'] = daily_tmax_df['VALUE'].map(lambda x: int(x[0:-1])/10)
daily_tmax_df

In [ ]:
daily_temp_df = pd.merge(daily_tmin_df, daily_tmax_df, on=['STNID','MONTH','DAY'], suffixes=['_MIN','_MAX'])

In [ ]:
daily_temp_df.groupby('STNID')['VALUE_MIN'].quantile(.05)

In [ ]:
daily_temp_df.groupby('STNID')['VALUE_MAX'].quantile(.95)

In [ ]:
df = pd.DataFrame({'min_05': daily_temp_df.groupby('STNID')['VALUE_MIN'].quantile(.1), 'max_95':daily_temp_df.groupby('STNID')['VALUE_MAX'].quantile(.9)}).reset_index()

In [ ]:
df[(df['min_05'] > 20) & (df['max_95'] < 80)]

In [ ]:
all_stations_df = pd.read_fwf(f'{weather_dir}/allstations.txt', header=None)
all_stations_df.columns = ['ID', 'Latitude','Longitude','Elevation','State','Name','GSNFLAG','HCNFLAG','WMOID']
all_stations_df.head()

In [ ]:
stations = pd.merge(df[(df['min_05'] > 25) & (df['max_95'] < 85)],all_stations_df,left_on='STNID',right_on='ID')
stations

In [ ]:
stations['State'].value_counts()

In [ ]:
df['Date'] = pd.to_datetime(df[['Year','Month','Day','Hour']])
df.head()

In [ ]:
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode()

fig = go.Figure()

fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = stations['Longitude'],
    lat = stations['Latitude'],
    hoverinfo = 'text',
    text = f"{stations['min_05']} to {stations['max_95']}",
    mode = 'markers',
    marker = dict(
        size = 2,
        color = 'rgb(255, 0, 0)',
        line = dict(
            width = 3,
            color = 'rgba(68, 68, 68, 0)'
        )
    )))

fig.show()